# Whitening

See [notes](http://joelouismarino.github.io/blog_posts/blog_whitening.html).

In [11]:
import whitening

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
X = pd.read_csv('./../../../data_source/to_analysis/complete.csv', index_col=0)
X.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxis,original_shape_MajorAxis,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MinorAxis,original_shape_Sphericity,...,Cisplatin_1,Cisplatin_2,Cisplatin_3,Cisplatin_4,Cisplatin_5,Cisplatin_6,Stage 1,Stage 2,Stage 3,Stage 4
patient,,,,,,,,,,,,,,,,,,,,,
2,0.738882,0.723925,27.060529,37.380273,41.976184,44.598206,42.720019,45.617979,27.619612,0.661532,...,0,0,0,0,0,1,0,0,0,1
4,0.796900,0.629917,19.845151,31.504408,38.587563,35.468296,29.410882,38.704005,25.105855,0.701721,...,0,0,0,0,1,0,0,0,0,1
5,0.600926,0.535140,22.515072,42.073251,46.065171,43.011626,32.015621,46.454279,25.282894,0.762365,...,0,0,0,0,0,0,0,1,0,0
8,0.784571,0.414247,30.263897,73.057649,74.148500,80.956779,65.764732,83.438600,57.318945,0.520001,...,0,0,1,0,0,0,0,0,0,1
10,0.690320,0.539743,19.449801,36.035312,33.286634,38.013156,33.015148,43.150898,24.875896,0.643822,...,0,0,0,0,0,1,0,0,0,1


In [3]:
X.shape

(198, 241)

In [15]:
spherer = whitening.Whitening(method='zca_cor')
Z = spherer.fit_transform(X.values)

In [16]:
# Not of unit length.
np.sum(spherer.W[:, 0])

-86.40721044607287

In [17]:
Z.shape

(198, 241)

In [18]:
Z_df = pd.DataFrame(Z, columns=X.columns, index=X.index)
Z_df.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxis,original_shape_MajorAxis,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MinorAxis,original_shape_Sphericity,...,Cisplatin_1,Cisplatin_2,Cisplatin_3,Cisplatin_4,Cisplatin_5,Cisplatin_6,Stage 1,Stage 2,Stage 3,Stage 4
patient,,,,,,,,,,,,,,,,,,,,,
2,1.180350e+20,5.970761e+19,8.066112e+19,6.263770e+20,1.690987e+20,-3.370381e+19,3.495441e+19,-5.426959e+20,2.504952e+19,2.003773e+20,...,1.225908e+18,6.602469e+19,-7.358468e+18,-1.020879e+19,7.159228e+19,3.018803e+19,-4.128500e+19,1.557930e+20,1.642563e+20,1.143660e+20
4,1.372817e+20,6.907376e+19,9.426932e+19,7.237666e+20,1.976043e+20,-3.822897e+19,4.229921e+19,-6.295358e+20,2.701289e+19,2.355838e+20,...,1.394705e+18,7.628405e+19,-8.374682e+18,-1.192991e+19,8.245831e+19,3.467275e+19,-4.727151e+19,1.827607e+20,1.935721e+20,1.357961e+20
5,1.296498e+20,6.557123e+19,8.874064e+19,6.884694e+20,1.856193e+20,-3.724918e+19,3.806889e+19,-5.960282e+20,2.791020e+19,2.198286e+20,...,1.353279e+18,7.258291e+19,-8.082229e+18,-1.117915e+19,7.875872e+19,3.324169e+19,-4.540856e+19,1.708868e+20,1.800318e+20,1.252684e+20
8,6.675617e+19,3.461976e+19,4.490524e+19,3.672298e+20,9.292659e+19,-2.199210e+19,1.500084e+19,-3.110279e+20,2.024199e+19,1.063756e+20,...,7.936784e+17,3.870519e+19,-4.601678e+18,-5.579220e+18,4.278980e+19,1.836074e+19,-2.540847e+19,8.367471e+19,8.565340e+19,5.694874e+19
10,1.346776e+20,6.695285e+19,9.510753e+19,7.082981e+20,1.936462e+20,-3.870889e+19,4.032664e+19,-6.144720e+20,2.768763e+19,2.318235e+20,...,1.416244e+18,7.469879e+19,-7.888685e+18,-1.143308e+19,8.110919e+19,3.434207e+19,-4.607258e+19,1.789088e+20,1.891098e+20,1.333618e+20


In [19]:
Z_df.to_csv('./../../../data_source/to_analysis/zca_corr_complete.csv')